In [1]:
from selenium import webdriver

#chrome webdriver manager package: pip install webdriver-manager
from webdriver_manager.chrome import ChromeDriverManager


#options for headless mode etc.
from selenium.webdriver.chrome.options import Options

#for waiting till an element loads
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [2]:
from bs4 import BeautifulSoup
import requests
import docx             #for writing to word document
from PIL import Image   #for downloading/manipulating images
import time
from IPython.display import display #for displayin in jypyter notebooks
import io               #for screenshot (bytes to pillow)

import os               #remove temp files ex. images
#import glob             #remove temp files ex. images

In [3]:
location = '/Users/arunRB/Downloads/VELPARI_FILES/VELPARI_FULL/'
url_file = 'velpari_all_urls.txt'
book_file = 'Veerayuga_Naayagan_Velpari.docx'

In [4]:
# chrome_options = Options()
# chrome_options.headless = False
# #chrome_options.add_argument('--start-maximized')
# browser = webdriver.Chrome('/Users/arunRB/Downloads/chromedriver', options=chrome_options)
# browser.get('https://www.vikatan.com/news/celebrity/124661-vel-paari-historical-hero')

# timeout = 10 #seconds
# try:
#     element_present = EC.presence_of_element_located((By.CLASS_NAME, 'content-section'))
#     WebDriverWait(browser, timeout).until(element_present)
# except TimeoutException:
#     print('Timed out waiting for page to load')
#     browser.quit()

# if element_present:
#     soup_rand = BeautifulSoup(browser.page_source, 'html.parser')
#     browser.close()

In [5]:
class GetChapter:
    
    ## FUNCTION WRITE TO DOCX
    #create & write to docx
    #https://automatetheboringstuff.com/chapter13/     ==> go to “Writing Word Documents”
    def write_chapter_to_docx(self, story_elems, chapter_name):

        doc_name = location + book_file
        temp_path = location + 'TEMP_IMAGES/'

    
        doc = docx.Document(doc_name)
        
        print(f'writing chapter\n')
        
#         doc.add_paragraph(chapter_name, style='Intense Quote')
        
        p = doc.add_paragraph('')
        p.add_run(chapter_name).bold = True

        
        for elem in story_elems:
            #print(f'elem = {elem}')
            figure_elem = elem.find('img')
            if not figure_elem:
                doc.add_paragraph(elem.text)
            else:
                img_url = figure_elem['data-src']
                #download img to a pillow object
                pil_img = Image.open(requests.get(img_url, stream = True).raw)
                local_filename = temp_path + img_url.split('/')[-1].split("?")[0]
                pil_img.save(local_filename)

                doc.add_picture(local_filename)

                #save local img paths
                #saved_imgs.append(local_filename)


        #display(doc_name)
        doc.add_paragraph('')
        doc.add_paragraph('')
        doc.add_page_break()
        doc.save(doc_name)
        
        print(f'### Chapter Written Successfully!! : {chapter_name}\n')
        print(f'------------------------------------------------------------------------\n')

        rm_all_temp_path = 'rm -rf '+ temp_path + '*'
        #display(rm_all_temp_path)
        
        #os.system(rm_all_temp_path)
        
    
    ## FUNCTION
    # given a url & webdriver instance do the following
    #    -get url in (headless mode) 
    #    -wait till content-section is loaded(WebDriverWait) or throw exception after timeout
    #    -crete a BS4 object from the successfully loaded page & close "tab"??
    #    -use css selector to get all the story emements(both text & images)
    #    -create & write story elements to docx
    #    -save image elements to a local temp folder

    
    def fetch_chapter(self, browser, url):
        if not browser:
            print("\n\n#### issue with chrome instance, exititng!!!")

        browser.get(url)
        timeout = 20 #seconds

        try:
            element_present = EC.presence_of_element_located((By.CLASS_NAME, 'content-section'))
            WebDriverWait(browser, timeout).until(element_present)
        except TimeoutException:
            print('Timed out waiting for page to load')
            #browser.quit()
            
            print(f'Try to bypass paywall..')
            

        if element_present:
            soup_rand = BeautifulSoup(browser.page_source, 'html.parser')
            browser.close()

            chapter_name = soup_rand.h1.text
            print(f'loaded chapter: {chapter_name}')
            #print(type(soup_rand)

            #use css selector to get all the story emements(both text & images)
            story_elems = soup_rand.select("[class~=story-element]")
            
            if not story_elems:
                print(f'Did not read content!! len story_elems= {len(story_elems)}')
                return False
            else:
                #write chapter as doc
                self.write_chapter_to_docx(story_elems,chapter_name)
            
                return True
        else:
            soup_rand = BeautifulSoup(browser.page_source, 'html.parser')
            browser.close()
            display(soup_rand)
            return False

In [6]:
def main():
    display("Hello World!")
    urls = []
    
    with open(location + url_file) as f:
        urls = [line for line in f]
    
    
    chrome_driver_path = '/Users/arunRB/Downloads/chromedriver'
    
    chrome_options = Options()
    chrome_options.headless = False
    #chrome_options.add_argument('--start-maximized')
    
    #manually using local chromedriver path
    #browser = webdriver.Chrome(chrome_driver_path, options=chrome_options)
    
    #browser = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)
    
    i = 70
    retry_count = 3
    while i < len(urls):
        url = urls[i]
        print(f'processing url {i}: {url}')
        browser = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)
        
        get_chapter_instance = GetChapter()
        was_success = get_chapter_instance.fetch_chapter(browser, url)
        
        if was_success:
            i = i + 1
            retry_count = 3
        elif retry_count >= 0:
            retry_count =retry_count -1 
            print(f" ### RETRYING {retry_count}")
        else:
            print(f'########## ABORTING. RETRY FAILED FOR URL {url} ##########')
            break

if __name__ == "__main__":
    main()


'Hello World!'

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [/Users/arunRB/.wdm/drivers/chromedriver/mac64/87.0.4280.88/chromedriver] found in cache


processing url 70: https://www.vikatan.com/arts/literature/138600-vel-paari-historical-hero

 
loaded chapter: வீரயுக நாயகன் வேள்பாரி - 70
writing chapter



[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [/Users/arunRB/.wdm/drivers/chromedriver/mac64/87.0.4280.88/chromedriver] found in cache


### Chapter Written Successfully!! : வீரயுக நாயகன் வேள்பாரி - 70

------------------------------------------------------------------------

processing url 71: https://www.vikatan.com/arts/literature/138807-vel-paari-historical-hero

 
loaded chapter: வீரயுக நாயகன் வேள்பாரி - 71
writing chapter



...

### Chapter Written Successfully!! : வீரயுக நாயகன் வேள்பாரி - 109

------------------------------------------------------------------------

processing url 110: https://www.vikatan.com/arts/literature/146129-vel-paari-historical-hero

 
loaded chapter: வீரயுக நாயகன் வேள்பாரி - 110
writing chapter



[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280


### Chapter Written Successfully!! : வீரயுக நாயகன் வேள்பாரி - 110

------------------------------------------------------------------------

processing url 111: https://www.vikatan.com/arts/literature/146278-vel-paari-historical-hero

 


[WDM] - Driver [/Users/arunRB/.wdm/drivers/chromedriver/mac64/87.0.4280.88/chromedriver] found in cache


loaded chapter: வீரயுக நாயகன் வேள்பாரி - 111
writing chapter

### Chapter Written Successfully!! : வீரயுக நாயகன் வேள்பாரி - 111

------------------------------------------------------------------------

